## Finding locations of jamcams in ~ULEZ
23 Aug 2019, Jack Hensley

In [42]:
import folium
import boto3
import pandas as pd
import operator
import numpy as np
from scipy.spatial.distance import cdist
import json
from shapely.geometry import shape, Point

% matplotlib inline

UsageError: Line magic function `%` not found.


import jamcams locations and see what is near center of london

In [43]:
# import jam cam locations
session = boto3.Session(profile_name='dssg')
s3 = session.client('s3')
bucket_name = 'air-pollution-uk'
obj = s3.get_object(Bucket=bucket_name, Key='processed_data/jamcams/jamcam_coords.csv')
jc_df = pd.read_csv(obj['Body'])

jc_df.head()

,NAME,LATITUDE,LONGITUDE
0,Romford_Rd/Tennyson_Rd,51.5421,0.00524
1,Piccadilly_Circus,51.5096,-0.13484
2,Blackheath_Rd/Greenwich_High_Rd,51.4742,-0.02073
3,Edgware_Way/Broadfields_Ave,51.6216,-0.27384
4,Cromwell_Rd/Earls_Court_Rd,51.4946,-0.19570


In [44]:
with open('C:/Users/joh3146/Downloads/londoncongestionchargezone.json') as f:
    ulezdata = json.load(f)
    
    
ulezdata

{'type': 'FeatureCollection',
 'crs': {'type': 'name',
  'properties': {'name': 'urn:ogc:def:crs:OGC:1.3:CRS84'}},
 'features': [{'type': 'Feature',
   'properties': {'BOUNDARY': 'CSS Area 1'},
   'geometry': {'type': 'Polygon',
    'coordinates': [[[-0.101771, 51.530696],
      [-0.100871, 51.530459],
      [-0.10087, 51.530448],
      [-0.100861, 51.530442],
      [-0.100604, 51.530368],
      [-0.100052, 51.53023],
      [-0.099106, 51.529982],
      [-0.099073, 51.530003],
      [-0.098725, 51.529906],
      [-0.098478, 51.529842],
      [-0.098311, 51.529779],
      [-0.097738, 51.529634],
      [-0.096784, 51.529394],
      [-0.096806, 51.52936],
      [-0.096615, 51.529313],
      [-0.096549, 51.529297],
      [-0.095942, 51.529145],
      [-0.094844, 51.528867],
      [-0.09412, 51.528683],
      [-0.093791, 51.528599],
      [-0.093284, 51.528472],
      [-0.092818, 51.528354],
      [-0.092639, 51.528316],
      [-0.092508, 51.528308],
      [-0.092083, 51.528203],
      [-0.

the desire is to have one detector nodes within central london that could possibly have a jamcam nearby.

In [45]:
central_london_easting = 530034
central_london_northing = 180381

df of tims locations in central london with only one detector per node selected. let's see if they're near jam cams.

plot on city of london map

find jamcams that are in similar locations

In [46]:
jc_files = pd.read_json('C:/Users/joh3146/Downloads/cam_file.json')
jc_df = jc_files.T

list(jc_df.columns.values)

['$type',
 'additionalProperties',
 'children',
 'childrenUrls',
 'commonName',
 'id',
 'lat',
 'lon',
 'placeType',
 'url']

In [47]:
jc_df.id

JamCams_00001.02151    JamCams_00001.02151
JamCams_00001.07450    JamCams_00001.07450
JamCams_00001.03675    JamCams_00001.03675
JamCams_00001.09747    JamCams_00001.09747
JamCams_00001.06600    JamCams_00001.06600
JamCams_00001.08858    JamCams_00001.08858
JamCams_00001.03758    JamCams_00001.03758
JamCams_00001.02900    JamCams_00001.02900
JamCams_00001.04235    JamCams_00001.04235
JamCams_00001.04353    JamCams_00001.04353
JamCams_00002.00635    JamCams_00002.00635
JamCams_00001.04568    JamCams_00001.04568
JamCams_00001.07929    JamCams_00001.07929
JamCams_00001.06649    JamCams_00001.06649
JamCams_00001.03762    JamCams_00001.03762
JamCams_00001.06519    JamCams_00001.06519
JamCams_00001.04214    JamCams_00001.04214
JamCams_00001.04653    JamCams_00001.04653
JamCams_00001.06511    JamCams_00001.06511
JamCams_00001.03757    JamCams_00001.03757
JamCams_00001.03819    JamCams_00001.03819
JamCams_00001.04640    JamCams_00001.04640
JamCams_00001.04565    JamCams_00001.04565
JamCams_000

In [48]:
jc_df = jc_df.rename(index=str, columns={"lat": "LATITUDE", "lon": "LONGITUDE"})

In [49]:
jc_locations = jc_df[['LATITUDE', 'LONGITUDE']]
jc_location_list = jc_locations.values.tolist()
jc_location_list[0:10]

[[51.5421, 0.005240000000000001],
 [51.5096, -0.13484000000000002],
 [51.4742, -0.020730000000000002],
 [51.6216, -0.27384000000000003],
 [51.4946, -0.1957],
 [51.514, -0.15409],
 [51.441, -0.10633000000000001],
 [51.5336, -0.02256],
 [51.4949, -0.12891],
 [51.3741, -0.09646]]

In [50]:
London = [51.506949, -0.122876]
londonmap = folium.Map(
    width=500, height=500,
    location = London,
    zoom_start = 12, 
    tiles = 'stamentoner')

for point in range(0, len(jc_location_list)):
    folium.CircleMarker(jc_location_list[point],
                        radius=1,
                        color='red',
                        fill_color='red',
                        fill_opacity=0.2
                       ).add_to(londonmap)

    
folium.GeoJson(
    ulezdata,
    name='geojson'
).add_to(londonmap)


londonmap

find cameras in the ulez

In [83]:
jc_df.LONGITUDE.values[0]

0.005240000000000001

In [119]:
jc_points = []
jc_points_idx = []
for tick in range(len(jc_df)):
    latlon = (jc_df.LONGITUDE.values[tick], jc_df.LATITUDE.values[tick])
    jc_point = Point(latlon)
    jc_points.append(jc_point)
    jc_points_idx.append(tick)

In [130]:
# check each polygon to see if it contains the point
jc_in_ulez = []
jc_in_ulez_idx = []
for feature in ulezdata['features']:
    polygon = shape(feature['geometry'])
    count = 0
    for jamcam in jc_points:
        if polygon.contains(jamcam):
            jc_in_ulez.append(jamcam)
            jc_in_ulez_idx.append(count)
        count += 1

In [131]:
jc_in_ulez_idx

[1,
 5,
 8,
 16,
 28,
 34,
 61,
 76,
 127,
 180,
 193,
 197,
 200,
 205,
 217,
 256,
 260,
 270,
 277,
 323,
 326,
 356,
 368,
 376,
 396,
 418,
 443,
 448,
 471,
 501,
 516,
 523,
 539,
 558,
 569,
 570,
 571,
 588,
 589,
 592,
 600,
 602,
 631,
 632,
 633,
 651,
 668,
 686,
 709,
 711,
 728,
 751,
 760,
 770,
 781,
 788,
 802,
 804,
 812,
 816,
 838,
 872,
 874,
 881,
 908]

In [137]:
jc_ulez_df = jc_df.iloc[jc_in_ulez_idx]
jc_ulez_df.head()

,$type,additionalProperties,children,childrenUrls,commonName,id,LATITUDE,LONGITUDE,placeType,url
JamCams_00001.07450,"Tfl.Api.Presentation.Entities.Place, Tfl.Api.P...",[{'$type': 'Tfl.Api.Presentation.Entities.Addi...,[],[],Piccadilly Circus,JamCams_00001.07450,51.5096,-0.13484,JamCam,/Place/JamCams_00001.07450
JamCams_00001.08858,"Tfl.Api.Presentation.Entities.Place, Tfl.Api.P...",[{'$type': 'Tfl.Api.Presentation.Entities.Addi...,[],[],Oxford St/Orchard St,JamCams_00001.08858,51.514,-0.15409,JamCam,/Place/JamCams_00001.08858
JamCams_00001.04235,"Tfl.Api.Presentation.Entities.Place, Tfl.Api.P...",[{'$type': 'Tfl.Api.Presentation.Entities.Addi...,[],[],Horseferry Rd/Marsham St,JamCams_00001.04235,51.4949,-0.12891,JamCam,/Place/JamCams_00001.04235
JamCams_00001.04214,"Tfl.Api.Presentation.Entities.Place, Tfl.Api.P...",[{'$type': 'Tfl.Api.Presentation.Entities.Addi...,[],[],Southwark St/Southwark Bdge Rd,JamCams_00001.04214,51.5048,-0.09535,JamCam,/Place/JamCams_00001.04214
JamCams_00001.03600,"Tfl.Api.Presentation.Entities.Place, Tfl.Api.P...",[{'$type': 'Tfl.Api.Presentation.Entities.Addi...,[],[],Farringdon Rd opp Ray St,JamCams_00001.03600,51.5232,-0.10792,JamCam,/Place/JamCams_00001.03600


In [142]:
jc_ulez_df['cam_id'] = jc_ulez_df.id.apply(lambda x: x.split('_')[1])

C:\Users\joh3146\AppData\Local\Continuum\anaconda3\envs\dssg\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [144]:
jc_ulez_df['cam_id'].values

array(['00001.07450', '00001.08858', '00001.04235', '00001.04214',
       '00001.03600', '00001.03507', '00001.04328', '00001.02500',
       '00001.07550', '00001.04225', '00001.06507', '00001.04245',
       '00001.03601', '00001.06590', '00001.07300', '00001.08859',
       '00001.02002', '00001.04680', '00001.03608', '00001.02100',
       '00001.06592', '00001.07590', '00001.03490', '00001.06510',
       '00001.07400', '00001.06503', '00001.03505', '00001.07389',
       '00001.04250', '00001.07359', '00001.06591', '00001.01551',
       '00001.04254', '00001.06501', '00001.04220', '00001.03611',
       '00001.07551', '00001.07301', '00001.07382', '00001.04502',
       '00001.07451', '00001.07330', '00001.06504', '00001.07458',
       '00001.03610', '00001.09640', '00001.04601', '00001.04227',
       '00001.04255', '00001.07600', '00001.04252', '00001.03612',
       '00001.07452', '00001.04244', '00001.03488', '00001.03609',
       '00001.08860', '00001.07500', '00001.03590', '00001.089

In [133]:
jc_ulez_df.to_csv(path_or_buf='C:/Users/joh3146/Downloads/ULEZ_cams.csv', sep=',', index=False)

select of jc_df the indices corresponding to the 20 nearest neighbors and export to csv

In [134]:
jc_out_locations = jc_ulez_df[['LATITUDE', 'LONGITUDE']]
jc_out_location_list = jc_out_locations.values.tolist()
jc_out_location_list

[[51.5096, -0.13484000000000002],
 [51.514, -0.15409],
 [51.4949, -0.12891],
 [51.5048, -0.09535],
 [51.5232, -0.10792],
 [51.5046, -0.08342000000000001],
 [51.4897, -0.13145],
 [51.5107, -0.11512000000000001],
 [51.5113, -0.1283],
 [51.5043, -0.11464],
 [51.4966, -0.14156000000000002],
 [51.499, -0.10493000000000001],
 [51.5072, -0.10443000000000001],
 [51.5114, -0.11924000000000001],
 [51.5157, -0.12612],
 [51.5144, -0.1494],
 [51.5095, -0.08688000000000001],
 [51.5011, -0.09317],
 [51.52, -0.10586000000000001],
 [51.5144, -0.07411000000000001],
 [51.5078, -0.14052],
 [51.5191, -0.12116],
 [51.4968, -0.08187000000000001],
 [51.4987, -0.11242],
 [51.5176, -0.14152],
 [51.5054, -0.14559000000000002],
 [51.5039, -0.08136],
 [51.5239, -0.1341],
 [51.4985, -0.11747],
 [51.5264, -0.12967],
 [51.5078, -0.13134],
 [51.5246, -0.111],
 [51.4902, -0.12259],
 [51.5011, -0.12628],
 [51.5069, -0.10845],
 [51.498, -0.10376],
 [51.5131, -0.12914],
 [51.5164, -0.13022],
 [51.5229, -0.12584],
 [51.500

In [136]:
London = [51.506949, -0.122876]
londonmap = folium.Map(
    width=500, height=500,
    location = London,
    zoom_start = 12, 
    tiles = 'stamentoner')
    
for point in range(0, len(jc_out_location_list)):
    folium.CircleMarker(jc_out_location_list[point],
                        radius=5,
                        color='blue',
                        fill_color='blue',
                        fill_opacity=0
                       ).add_to(londonmap)
    
folium.GeoJson(
    ulezdata,
    name='geojson'
).add_to(londonmap)
    
londonmap.save("C:/Users/joh3146/Documents/dssg/jamcams_tims_selected_for_eval.html")
londonmap